In [ ]:
# If running in a new environment, uncomment the following lines to install dependencies:

# !pip install torch torchvision transformers opencv-python pillow matplotlib

import cv2

import numpy as np

import torch

from transformers import Mask2FormerForUniversalSegmentation, AutoImageProcessor

from PIL import Image

import matplotlib.pyplot as plt

import os

from google.colab import files



# --- PatternDetector for optimal tile pattern ---

class PatternDetector:

    def __init__(self, tile_image):

        self.original_tile = tile_image

        self.height, self.width = tile_image.shape[:2]



    def analyze_optimal_pattern(self):

        candidates = {

            'standard': self._create_2x2_standard,

            'rotated_quadrant': self._create_2x2_rotated,

            'mirrored': self._create_2x2_mirrored,

            'diamond': self._create_2x2_diamond

        }

        best_score = float('inf')

        best_pattern_name = 'standard'

        best_composite = None

        for name, method in candidates.items():

            composite = method()

            score = self._calculate_seam_error(composite)

            if score < best_score:

                best_score = score

                best_pattern_name = name

                best_composite = composite

        standard_score = self._calculate_seam_error(self._create_2x2_standard())

        if best_pattern_name != 'standard' and best_score > standard_score * 0.8:

            best_pattern_name = 'standard'

            best_composite = self._create_2x2_standard()

        return best_composite, best_pattern_name

    def _create_2x2_standard(self):

        top_row = np.hstack([self.original_tile, self.original_tile])

        return np.vstack([top_row, top_row])

    def _create_2x2_rotated(self):

        tl = self.original_tile

        tr = cv2.rotate(self.original_tile, cv2.ROTATE_90_CLOCKWISE)

        br = cv2.rotate(self.original_tile, cv2.ROTATE_180)

        bl = cv2.rotate(self.original_tile, cv2.ROTATE_90_COUNTERCLOCKWISE)

        return np.vstack([np.hstack([tl, tr]), np.hstack([bl, br])])

    def _create_2x2_mirrored(self):

        tile = self.original_tile

        tile_hflip = cv2.flip(tile, 1)

        tile_vflip = cv2.flip(tile, 0)

        tile_hvflip = cv2.flip(tile, -1)

        return np.vstack([np.hstack([tile, tile_hflip]), np.hstack([tile_vflip, tile_hvflip])])

    def _create_2x2_diamond(self):

        tile = self.original_tile

        rot45 = cv2.warpAffine(tile, cv2.getRotationMatrix2D((self.width//2, self.height//2), 45, 1.0), (self.width, self.height))

        return np.vstack([np.hstack([tile, rot45]), np.hstack([rot45, tile])])

    def _calculate_seam_error(self, composite):

        # Simple seam error: sum of absolute differences along seams

        h, w = composite.shape[:2]

        seam_v = np.sum(np.abs(composite[:, w//2-1] - composite[:, w//2]))

        seam_h = np.sum(np.abs(composite[h//2-1, :] - composite[h//2, :]))

        return seam_v + seam_h



# --- RealisticBlending for color/brightness matching and blending ---

class RealisticBlending:

    def __init__(self, original_image, mask):

        self.original_image = original_image.copy()

        self.mask = mask

        self.height, self.width = original_image.shape[:2]

        self._compute_floor_statistics()

    def _compute_floor_statistics(self):

        floor_region = cv2.bitwise_and(self.original_image, self.original_image, mask=self.mask)

        floor_pixels = floor_region[self.mask > 0]

        if len(floor_pixels) > 0:

            self.floor_mean_color = np.mean(floor_pixels, axis=0)

            self.floor_std_color = np.std(floor_pixels, axis=0)

            self.floor_mean_brightness = np.mean(cv2.cvtColor(floor_region, cv2.COLOR_BGR2GRAY)[self.mask > 0])

        else:

            self.floor_mean_color = np.array([128, 128, 128])

            self.floor_std_color = np.array([30, 30, 30])

            self.floor_mean_brightness = 128

    def match_brightness(self, tile_image, target_brightness=None, strength=0.2):

        if target_brightness is None:

            target_brightness = self.floor_mean_brightness

        tile_gray = cv2.cvtColor(tile_image, cv2.COLOR_BGR2GRAY)

        tile_brightness = np.mean(tile_gray[tile_gray > 0])

        if tile_brightness == 0:

            return tile_image

        brightness_factor = 1.0 + (target_brightness / tile_brightness - 1.0) * strength

        adjusted = tile_image.astype(np.float32) * brightness_factor

        adjusted = np.clip(adjusted, 0, 255).astype(np.uint8)

        return adjusted

    def blend(self, warped_tiles, alpha=0.95, feather_size=3):

        mask_blur = cv2.GaussianBlur(self.mask, (feather_size, feather_size), 0) / 255.0

        mask_3 = np.stack([mask_blur] * 3, axis=-1)

        result = (self.original_image.astype(np.float32) * (1 - mask_3) + warped_tiles.astype(np.float32) * mask_3).astype(np.uint8)

        return result



# --- Main workflow ---

# Load Mask2Former model and processor

model_name = "facebook/mask2former-swin-base-ade-semantic"

processor = AutoImageProcessor.from_pretrained(model_name)

model = Mask2FormerForUniversalSegmentation.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

model.eval()



print("Upload ROOM image:")

uploaded = files.upload()

room_image_path = list(uploaded.keys())[0]

image_pil = Image.open(room_image_path).convert("RGB")

image = image_pil

inputs = processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():

    outputs = model(**inputs)

result = processor.post_process_semantic_segmentation(

    outputs,

    target_sizes=[image.size[::-1]]

)[0]

segmentation_map = result.cpu().numpy()



# Use ADE20K class IDs: 0 = wall, 3 = floor

WALL_ID = 0

FLOOR_ID = 3

wall_mask = (segmentation_map == WALL_ID).astype(np.uint8) * 255

floor_mask = (segmentation_map == FLOOR_ID).astype(np.uint8) * 255



plt.figure(figsize=(15,5))

plt.subplot(1,3,1)

plt.imshow(image_pil)

plt.title("Original")

plt.axis("off")

plt.subplot(1,3,2)

plt.imshow(wall_mask, cmap="gray")

plt.title("Wall Mask")

plt.axis("off")

plt.subplot(1,3,3)

plt.imshow(floor_mask, cmap="gray")

plt.title("Floor Mask")

plt.axis("off")

plt.show()



print("Upload TILE image:")

uploaded_tile = files.upload()

tile_image_path = list(uploaded_tile.keys())[0]

tile_img_pil = Image.open(tile_image_path).convert("RGB")

tile_img_np = np.array(tile_img_pil)

tile_img_cv = cv2.cvtColor(tile_img_np, cv2.COLOR_RGB2BGR)



# --- Pattern detection for optimal macro-tile ---

pattern_detector = PatternDetector(tile_img_cv)

macro_tile, pattern_name = pattern_detector.analyze_optimal_pattern()

if pattern_name != 'standard':

    tile_img_cv = macro_tile



# --- Get floor corners for perspective mapping ---

def order_points_clockwise(pts):

    rect = np.zeros((4,2), dtype="float32")

    s = pts.sum(axis=1)

    rect[0] = pts[np.argmin(s)]  # top-left

    rect[2] = pts[np.argmax(s)]  # bottom-right

    diff = np.diff(pts, axis=1)

    rect[1] = pts[np.argmin(diff)]  # top-right

    rect[3] = pts[np.argmax(diff)]  # bottom-left

    return rect

def get_floor_corners(mask):

    mask = (mask > 0).astype(np.uint8) * 255

    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:

        return None

    cnt = max(contours, key=cv2.contourArea)

    hull = cv2.convexHull(cnt)

    epsilon = 0.02 * cv2.arcLength(hull, True)

    approx = cv2.approxPolyDP(hull, epsilon, True)

    if len(approx) < 4:

        return None

    pts = approx.reshape(-1, 2)

    if len(pts) > 4:

        rect = cv2.minAreaRect(hull)

        pts = cv2.boxPoints(rect)

    pts = order_points_clockwise(pts)

    return np.float32(pts)



# --- Professional tile installation and blending ---

def professional_tile_installation(original_img, mask, tile_img, tile_size=200, grout=3, rotation_angle=0):

    H, W = original_img.shape[:2]

    mask = (mask > 0).astype(np.uint8) * 255

    dst_pts = get_floor_corners(mask)

    if dst_pts is None:

        print("Could not detect proper floor corners")

        return original_img

    tex_scale = 800

    src_pts = np.float32([

        [0, 0],

        [tex_scale, 0],

        [tex_scale, tex_scale],

        [0, tex_scale]

    ])

    # Create tile pattern with grout

    def create_tile_pattern(width, height, tile_img, tile_size=tile_size, grout=grout):

        grout_color = 220

        pattern = np.ones((height, width, 3), dtype=np.uint8) * grout_color

        tile_resized = cv2.resize(tile_img, (tile_size-grout, tile_size-grout))

        for y in range(0, height, tile_size):

            for x in range(0, width, tile_size):

                variation = 0.95 + 0.1 * np.random.rand()

                tile_var = np.clip(tile_resized * variation, 0, 255).astype(np.uint8)

                y_end = min(y+tile_size-grout, height)

                x_end = min(x+tile_size-grout, width)

                pattern[y:y_end, x:x_end] = tile_var[:y_end-y, :x_end-x]

        return pattern

    tile_pattern = create_tile_pattern(tex_scale, tex_scale, tile_img, tile_size=tile_size, grout=grout)

    center = (tex_scale / 2, tex_scale / 2)

    rot_mat = cv2.getRotationMatrix2D(center, rotation_angle, 1.0)

    rotated_src_pts = cv2.transform(src_pts.reshape(-1, 1, 2), rot_mat).reshape(-1, 2)

    H_matrix = cv2.getPerspectiveTransform(rotated_src_pts, dst_pts)

    warped_tiles = cv2.warpPerspective(tile_pattern, H_matrix, (W, H), flags=cv2.INTER_LANCZOS4)

    # --- Realistic blending: brightness match and soft mask ---

    blending = RealisticBlending(original_img, mask)

    warped_tiles = blending.match_brightness(warped_tiles, strength=0.3)

    result = blending.blend(warped_tiles, alpha=0.95, feather_size=3)

    return result



# Convert PIL image to OpenCV BGR

image_cv = cv2.cvtColor(np.array(image_pil), cv2.COLOR_RGB2BGR)



# Apply professional tile installation to detected floor

result_tiled = professional_tile_installation(image_cv, floor_mask, tile_img_cv, tile_size=200, grout=3, rotation_angle=20)



plt.figure(figsize=(12,8))

plt.imshow(cv2.cvtColor(result_tiled, cv2.COLOR_BGR2RGB))

plt.axis("off")

plt.title("Tile Applied to Floor (Professional Realism)")

plt.show()



# Save result to outputs directory

os.makedirs("outputs", exist_ok=True)

result_path = os.path.join("outputs", "tiled_result.png")

cv2.imwrite(result_path, result_tiled)

print(f"✅ Tiled result saved to: {result_path}")



# Optional: Side-by-side comparison

side_by_side = np.hstack([

    cv2.cvtColor(image_cv, cv2.COLOR_BGR2RGB),

    cv2.cvtColor(result_tiled, cv2.COLOR_BGR2RGB)

])

plt.figure(figsize=(18,8))

plt.imshow(side_by_side)

plt.axis("off")

plt.title("Original vs. Tiled Result")

plt.show()